In [1]:
import os, nltk
import requests
import io #codecs
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk import word_tokenize, sent_tokenize, bigrams,trigrams
import pandas as pd
import codecs, re
if os.path.isfile('balochi.txt'):
    with io.open('balochi.txt', encoding='utf8') as fin:
        text = fin.read()
else:
    print('file not found')

len(text.split())


317181

In [2]:
 %pip install --upgrade nltk
 import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# Train on 80% f the corpus and test on the rest
split = int(80*len(text)/100)
train = text[:split]
test = text[split:]

In [4]:
print(len(train.split()))
print(len(test.split()))

253652
63530


In [5]:
words = text.split('۔')
avg = sum(map(len, words))/len(words)
avg

41.91805200012337

In [6]:
#training text
sentences = train.split("۔")
tokenized_text = [sent.translate(str.maketrans('', '', '''"'=+{}()[]/\|?;:-._!@#$%^,&*<>؟':؛،”“"…‘0123456789’''')) for sent in sentences]
tokenized_train = [word_tokenize(i) for i in tokenized_text]
tokenized_train

[['بچار',
  'خان',
  'صاحب',
  'میر',
  'احمد',
  'یار',
  'خان',
  'مئے',
  'ءُ',
  'ھاکم',
  'ءُ',
  'مئے',
  'سر',
  'ءِ',
  'ساھگ',
  'انت'],
 ['بلے', 'منی', 'گمان', 'انت'],
 ['اے',
  'نیمگ',
  'ءَ',
  'کدی',
  'در',
  'نہ',
  'روت',
  'ءُ',
  'نیکہ',
  'من',
  'ءَ',
  'اے',
  'سُج',
  'اِیت'],
 ['ءُ', 'نیکہ', 'منی', 'دل', 'گْوش', 'اِیت'],
 ['کہ',
  'خان',
  'صاحب',
  'میر',
  'احمد',
  'یار',
  'خان',
  'مئے',
  'خان',
  'ءُ',
  'مئے',
  'ھاکم',
  'ءُ',
  'سر',
  'ءِ',
  'ساھگ',
  'چشیں',
  'ردیں',
  'پیسلہ',
  'یے',
  'کنت'],
 ['میر',
  'مُراد',
  'جان',
  'گچکی',
  'چلیم',
  'ءِ',
  'دو',
  'سے',
  'دھوت',
  'گران',
  'چلیم',
  'ءَ',
  'وتی',
  'گہار',
  'ھانی',
  'ءِ',
  'نیمگ',
  'ءَ',
  'تیلانک',
  'دات'],
 ['کہ',
  'آئی',
  'گہار',
  'ھانی',
  'چشکہ',
  'پہ',
  'چلیم',
  'ءِ',
  'یک',
  'دھوت',
  'ءُ',
  'یک',
  'باپے',
  'ءَ',
  'ھدوناک',
  'بہ',
  'بیت'],
 ['چلیم', 'چہ', 'سرین', 'ءَ', 'گپت', 'ءُ', 'وتی', 'دیم', 'ءَ', 'ایر', 'کُرت'],
 ['ءُ',
  'چار',
  'زانو',
  'وتی',
  '

In [7]:
#testing text
sentences2 = test.split("۔")
tokenized_text2 = [sent.translate(str.maketrans('', '', '''"'=+{}()[]/\|?;:-._!@#$%^,&*<>؟':؛،”“"…‘0123456789’''')) for sent in sentences2]
tokenized_test = [word_tokenize(i) for i in tokenized_text2]
# tokenized_test

In [8]:
from nltk.lm.preprocessing import padded_everygram_pipeline
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_train)
test_data, padded_test = padded_everygram_pipeline(n, tokenized_test)

In [9]:
from nltk.lm import Laplace, KneserNeyInterpolated, Lidstone
model = Lidstone(0.5, 1) # training a 3-grams model
# model = Laplace(1)

In [10]:
#Initializing the Laplace model, creates an empty vocabulary
len(model.vocab)

0

In [11]:
#which gets filled as we fit the model.
model.fit(train_data, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 10258 items>


In [12]:
#number of unigrams, bigrams, trigrams, 4-grams and so on...
print(model.counts)

<NgramCounter with 3 ngram orders and 993957 ngrams>


In [13]:
s = 0
for k, t in enumerate(test_data):
    p = model.perplexity(t)
    s += p
print ("Perplexity: {0}".format(s/(k+1)))

Perplexity: 313.39956805182385


In [16]:
from collections import Counter
from itertools import chain
from bidi.algorithm import get_display


bigram_fd = Counter(chain.from_iterable(zip(sublist, sublist[1:]) for sublist in model.vocab.lookup(tokenized_train)))
bigram_df= pd.DataFrame(bigram_fd.most_common(10),columns=['bigrams', 'bigram_count'])


bigram_df

,bigrams,bigram_count
0,"(تہ, ءَ)",3563
1,"(ءِ, تہ)",2415
2,"(سر, ءَ)",1146
3,"(کہ, آ)",1037
4,"(بوتگ, اَت)",888
5,"(وت, ءَ)",692
6,"(ءَ, یک)",672
7,"(ءِ, سر)",633
8,"(نہ, اَت)",615
9,"(لوٹ, اِت)",613


In [15]:
pip install python-bidi


In [17]:
counts = Counter()   # or nltk.FreqDist()
for sent in model.vocab.lookup(tokenized_train):
    counts.update(nltk.ngrams(sent, 1))
trigram_df= pd.DataFrame(counts.most_common(10),columns=['trigram', 'trigram_count'])
trigram_df

,trigram,trigram_count
0,"(ءَ,)",19942
1,"(ءُ,)",16627
2,"(ءِ,)",12043
3,"(کہ,)",8233
4,"(اَت,)",7628
5,"(وتی,)",4397
6,"(آ,)",3823
7,"(تہ,)",3591
8,"(یک,)",3166
9,"(نہ,)",3086


In [18]:
counts = Counter()   # or nltk.FreqDist()
for sent in model.vocab.lookup(tokenized_train):
    counts.update(nltk.ngrams(sent, 4))
trigram_df= pd.DataFrame(counts.most_common(20),columns=['trigram', 'trigram_count'])
trigram_df

,trigram,trigram_count
0,"(دنیا, ءِ, تہ, ءَ)",165
1,"(ءِ, تہ, ءَ, یک)",154
2,"(دل, ءِ, تہ, ءَ)",112
3,"(آ, وت, ءَ, را)",111
4,"(کہ, ایشی, تہ, ءَ)",106
5,"(میر, احمد, یار, خان)",99
6,"(میر, مُراد, جان, گچکی)",93
7,"(اے, دنیا, ءِ, تہ)",91
8,"(ءَ, وت, ءَ, را)",90
9,"(ءِ, تہ, ءَ, آ)",84


In [19]:
counts = Counter()   # or nltk.FreqDist()
for sent in model.vocab.lookup(tokenized_train):
    counts.update(nltk.ngrams(sent, 5))
trigram_df= pd.DataFrame(counts.most_common(20),columns=['trigram', 'trigram_count'])
trigram_df

,trigram,trigram_count
0,"(اے, دنیا, ءِ, تہ, ءَ)",91
1,"(خان, میر, احمد, یار, خان)",58
2,"(وتی, دل, ءِ, تہ, ءَ)",52
3,"(میر, احمد, یار, خان, ءِ)",41
4,"(من, نہ, زان, آں, کہ)",36
5,"(تہ, ءَ, وت, ءَ, را)",35
6,"(کہ, آ, وت, ءَ, را)",27
7,"(خان, صاحب, میر, احمد, یار)",25
8,"(صاحب, میر, احمد, یار, خان)",25
9,"(ءِ, تہ, ءَ, وت, ءَ)",23


In [20]:
counts = Counter()   # or nltk.FreqDist()
for sent in model.vocab.lookup(tokenized_train):
    counts.update(nltk.ngrams(sent, 6))
trigram_df= pd.DataFrame(counts.most_common(20),columns=['trigram', 'trigram_count'])
# trigram_df

In [21]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.offline import iplot

# plotly express
fig = px.histogram(trigram_df, x='trigram_count', y='trigram', histfunc = 'sum')
fig.update_layout(
    title="Bigrams Histogram",
    xaxis_title="Bigrams",
    yaxis_title="Frequency of Bigrams",
    font=dict(
        size=14,
        color="RebeccaPurple"
    ))
# fig.update_yaxes(type="category")
fig.show()


In [22]:
import plotly.express as px
df = px.data.tips()
fig = px.bar(trigram_df, x="trigram_count", y="trigram", orientation='h')
fig.show()

In [23]:
fig = px.histogram(trigram_df, x='trigram', y='trigram_count', histfunc = 'sum')
fig.update_layout(
    title="Trigrams Histogram",
    xaxis_title="Trigrams",
    yaxis_title="Frequency of Trigrams",
    font=dict(
        size=14,
        color="RebeccaPurple"
    ))
fig.update_xaxes(type="category")
fig.show()

In [29]:
input_text = (input('Enter a word: ')).split()
sorted(model.counts[input_text].most_common(3))

Enter a word: آئی


[('دل', 91), ('وتی', 83), ('پِس', 73)]

In [27]:
import dill as pickle

with open('Balochi.pkl', 'wb') as fout:
    pickle.dump(model, fout)

In [28]:
pip install dill

ERROR: Operation cancelled by user


In [ ]:
from google.colab import drive
drive.mount('/content/drive')